In [32]:

import os

In [33]:
os.chdir("../")

In [34]:
%pwd

'/Users/pradhyumnyadav'

In [35]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Pradhyumn1/chest_ctscan_analysis.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Pradhyumn1"
os.environ["MLFLOW_TRACKING_PASSWORD"]="9668024cdf3cab6718afb606d69ebde8f71b7dcf"

In [36]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse



In [37]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [38]:
import os
import tensorflow as tf
from pathlib import Path

class EvaluationConfig:
    def __init__(self, path_of_model, training_data, all_params, mlflow_uri, params_image_size, params_batch_size):
        self.path_of_model = path_of_model
        self.training_data = training_data
        self.all_params = all_params
        self.mlflow_uri = mlflow_uri
        self.params_image_size = params_image_size
        self.params_batch_size = params_batch_size



In [39]:
class ConfigurationManager:
    def __init__(self):
        self.config = {
            "artifacts_root": "artifacts",
            "data_ingestion": {
                "root_dir": "artifacts/data_ingestion",
                "source_URL": "https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing",
                "local_data_file": "artifacts/data_ingestion/data.zip",
                "unzip_dir": "artifacts/data_ingestion"
            },
            "prepare_base_model": {
                "root_dir": "artifacts/prepare_base_model",
                "base_model_path": "artifacts/prepare_base_model/base_model.h5",
                "updated_base_model_path": "artifacts/prepare_base_model/base_model_updated.h5"
            },
            "training": {
                "root_dir": "artifacts/training",
                "trained_model_path": "artifacts/training/model.h5"
            }
        }
        self.params = {
            "AUGMENTATION": True,
            "IMAGE_SIZE": [224, 224, 3],
            "BATCH_SIZE": 16,
            "INCLUDE_TOP": False,
            "EPOCHS": 10,
            "CLASSES": 2,
            "WEIGHTS": "imagenet",
            "LEARNING_RATE": 0.01
        }

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/ct_scan",
            mlflow_uri="https://dagshub.com/Pradhyumn1/chest_ctscan_analysis.mlflow",
            all_params=self.params,
            params_image_size=self.params["IMAGE_SIZE"],
            params_batch_size=self.params["BATCH_SIZE"]
        )
        return eval_config


In [40]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def load_model(self, path):
        model_path = Path(path)
        if model_path.exists():
            self.model = tf.keras.models.load_model(model_path)
        else:
            raise FileNotFoundError(f"Model file not found at {model_path}")

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [42]:
try:
    config_manager = ConfigurationManager()
    eval_config = config_manager.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.load_model(eval_config.path_of_model)  # Pass the path argument here
    # Other evaluation steps here...
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An error occurred: {str(e)}")
